In [4]:
# imports
import requests
from Event import *
from IPython.display import Markdown, display

# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

system_prompt = """
You are an assistant working as an entity extractor. When one asks you to create an event for him, you extract entities as start datetime,
end datetime, summary, recurrence.
You answer only with a json built on the structure of a Google Calendar event. No explanation. No other words.
For example if prompt says:\n
dance lesson from 30 past ten pm to to 30 past eleven pm every Tuesday, Friday and Saturday\n
you respond:\n
{'summary': 'dance lesson', 'start': {'dateTime': "2025-02-07T22:30:00+02:00', 'timeZone':'Eastern European Time',"}, 'end': {'dateTime': "2025-06-30T23:30:00+02:00', 'timeZone':'Eastern European Time',"}, 'recurrence': ['RRULE:FREQ=WEEKLY;BYDAY=TU,FR,SA;UNTIL=20250630T000000Z']}
Start date of the event will be the first day after today.

"""

def getUserPrompt(user_prompt):
    prompt = "You are an assistant that creates Google Calendar events. Given this prompt:\n"
    prompt += user_prompt
    prompt += "\n you are supposed to return nothing else but a json formatted as a Google Calendar Event with the data given to the prompt."
    prompt += "\n If some crucial data is missing ask for it."
    return prompt

def getMessages(user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": getUserPrompt(user_prompt)}
    ]

def askOllama(user_prompt):
    MODEL = "Llama3.2"
    # first approach using requests library
    payload = {
        "model": MODEL,
        "messages": getMessages(user_prompt),
        "stream": False
    }
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    #print(response.json()['message']['content'])
    display(response.json()['message']['content'])



In [5]:
event = Event()
correct_response = event.create_event()
print(getUserPrompt(event.prompt['en']))
print(getUserPrompt(event.prompt['gr']))


You are an assistant that creates Google Calendar events. Given this prompt:
yoga class from 3 minutes to 4 until to 3 minutes to 5 Monday Thursday
 you are supposed to return nothing else but a json formatted as a Google Calendar Event with the data given to the prompt.
 If some crucial data is missing ask for it.
You are an assistant that creates Google Calendar events. Given this prompt:
μάθημα γιόγκα Δευτέρα Πέμπτη από τέσσερις παρά 3 -  ως τις πέντε παρά 3
 you are supposed to return nothing else but a json formatted as a Google Calendar Event with the data given to the prompt.
 If some crucial data is missing ask for it.


In [6]:
askOllama(event.prompt['gr'])

'{\n  "summary": "γιόγκα",\n  "start": {\n    "dateTime": "2025-02-08T15:00:00+02:00",\n    "timeZone": "Eastern European Time"\n  },\n  "end": {\n    "dateTime": "2025-06-30T17:03:00+02:00",\n    "timeZone": "Eastern European Time"\n  },\n  "recurrence": [\n    "RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;UNTIL=20250630T000000Z"\n  ]\n}'

In [7]:
display(correct_response['gr'])

{'summary': 'μάθημα γιόγκα',
 'start': {'dateTime': "2025-02-10T03:57:00+02:00', 'timeZone':'Eastern European Time',"},
 'end': {'dateTime': "2025-06-30T04:57:00+02:00', 'timeZone':'Eastern European Time',"},
 'recurrence': ['RRULE:FREQ=WEEKLY;BYDAY=MO,TH;UNTIL=20250630T000000Z']}

In [5]:
getOpenAIResponse(event.prompt['gr'])

'```json\n{\n  "start": {\n    "dateTime": "2023-11-11T11:36:00+00:00"\n  },\n  "end": {\n    "dateTime": "2023-11-11T12:36:00+00:00"\n  },\n  "summary": "Συνάντηση γονέων-καθηγητών",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'

In [6]:
getOpenAIResponse(event.prompt['en']) # 4-o

'```json\n{\n  "start": {\n    "dateTime": "2023-11-24T00:00:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-24T01:00:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Parent-Teacher Meeting",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'

In [7]:
getOpenAIResponse(event.prompt['gr'])

'```json\n{\n  "start": {\n    "dateTime": "2023-11-11T11:36:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-11T12:36:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Συνάντηση γονέων-καθηγητών",\n  "recurrence": "RRULE:FREQ=MONTHLY;BYMONTHDAY=11"\n}\n```'

In [8]:
getOpenAIResponse(event.prompt['en']) # o1

'```json\n{\n  "start": {\n    "dateTime": "2023-11-24T09:00:00",\n    "timeZone": "UTC"\n  },\n  "end": {\n    "dateTime": "2023-11-24T10:00:00",\n    "timeZone": "UTC"\n  },\n  "summary": "Parent-Teacher Meeting",\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'

In [9]:
getOpenAIResponse(event.prompt['gr'])

'```json\n{\n  "summary": "Συνάντηση γονέων-καθηγητών",\n  "start": {\n    "dateTime": "2023-11-11T11:36:00",\n    "timeZone": "Europe/Athens"\n  },\n  "end": {\n    "dateTime": "2023-11-11T12:36:00",\n    "timeZone": "Europe/Athens"\n  },\n  "recurrence": [\n    "RRULE:FREQ=MONTHLY;BYMONTHDAY=13"\n  ]\n}\n```'